In [ ]:
import pandas as pd
pd.set_option("display.max_rows", None)

In [ ]:
# For sentence tokenization
from nltk import tokenize
import pandas as pd

def read_paper(pid, section, source):
    with open(f'data/ebm_nlp_2_00/documents/{pid}.txt', 'r') as f:
        alltext = ''.join(f.readlines())
        sentences = tokenize.sent_tokenize(alltext)
        return pd.DataFrame({
            'URL': [f'https://pubmed.ncbi.nlm.nih.gov/{pid}'] * (2),
            'ID': [pid] * (2),
            'Type': ['Title', 'Abstract'],
            'Text': [sentences[0], ' '.join(sentences[1:])],
            'ann_section': [section] * (2),
            'ann_source': [source] * (2)
        })

read_paper(6989377, 'test', 'gold')

In [ ]:
# Build up the entire list of paper abstracts we have
import os

# directory = f'data/ebm_nlp_2_00/annotations/aggregated/hierarchical_labels/interventions/test/crowd'
# directory = f'data/ebm_nlp_2_00/annotations/aggregated/starting_spans/participants/test/crowd'
# directory = f'data/ebm_nlp_2_00/annotations/aggregated/starting_spans/outcomes/test/crowd'
# directory = f'data/ebm_nlp_2_00/annotations/aggregated/starting_spans/interventions/train/crowd'
# directory = f'data/ebm_nlp_2_00/annotations/aggregated/starting_spans/interventions/test/crowd'
directory = f'data/ebm_nlp_2_00/annotations/aggregated/hierarchical_labels/interventions/test/gold'
df = None

for filename in os.scandir(directory):
    # assume that the filepath does not contain .s except for the final extension!
#     print(filename.path)
#     print(filename.name.split('.')[0])
    df_file = read_paper(filename.name.split('.')[0], 'test', 'gold')
    # and append it to the summary dataframe
    if df is None:
        df = df_file
    else:
        df = df.append(df_file)

df = df.reset_index(drop=True)

df.to_pickle(f'temp/ebm-df.pkl')
df

In [ ]:
# Split the full-abstract CSV into a CSV containing individual sentences instead
def sentence_tokenize(group):
    row = group.reset_index(drop=True).loc[0]
    sentences = tokenize.sent_tokenize(row['Text'])
    return pd.DataFrame({
        'URL': [row['URL']] * (len(sentences)),
        'ID': [row['ID']] * (len(sentences)),
        'Type': [row['Type']] * (len(sentences)),
        'Index': list(range(len(sentences))),
        'Text': sentences,
        'ann_section': ['test'] * (len(sentences)),
        'ann_source': ['gold'] * (len(sentences))
    })

df_sentences = df.groupby(['ID', 'Type', 'ann_section', 'ann_source'], group_keys=False).apply(
    lambda group: sentence_tokenize(group)
).reset_index(drop=True)

df_sentences.to_pickle(f'temp/ebm-df_sentences.pkl')
df_sentences

In [ ]:
def extract_labeled_terms(tokens, tags):
    in_tag = False
    hits = []
    hit_indices = []
    sofar = []
    for i in range(len(tokens)):
        if int(tags[i]) != 0:
            if not in_tag:
                in_tag = True
            sofar.append(tokens[i])
        elif in_tag:
            in_tag = False
            hits.append(sofar)
            hit_indices.append(i - len(sofar))
            sofar = []
    if in_tag:
        hits.append(sofar)
        hit_indices.append(len(tokens) - len(sofar))
    return hits, hit_indices

test_extract_labeled_terms = extract_labeled_terms(
    ['This', 'is', 'an', 'example', 'sentence', '.'],
    ['1',    '0',  '0',  '1',       '1',        '0']
)
test_extract_labeled_terms

In [ ]:
# split_0 is the text literally preceding the anchor
# split_1 is the text that the anchor consists of
# split_2 is the text literally following the anchor
# split_tokens is the list of tokens that split identifies
# split_anchor_span is a tuple (anchor_first_token_index, anchor_last_token_index) or null if there is no anchor
# split_anchor_indices is a tuple (anchor_start_char_index, anchor_end_char_index) or null if there is no anchor
# split_anchor_type is unique to the EBM dataset - identifies what type of anchor we are using
splitting_headers = ['split_0','split_1','split_2', 
                     'split_tokens', 'split_anchor_span', 'split_anchor_indices', 
                     'within_anchor_index',
                     'split_anchor_type']
# Use columns ID, Type, Index to join with df_sentences
join_headers = ['ID', 'Type', 'Index', 'ann_section', 'ann_source']

In [ ]:
# Extract tokenization, part-of-speech tags from EBMNLP dataset
def ebmnlp_addtdata(group):
    group = group.sort_values(by=['Index'])
    group = group.sort_values(by=['Type'], ascending=False)
    group = group.reset_index()
    id_num = group.iloc[0]['ID']
    ann_section = group.iloc[0]['ann_section']
    ann_source = group.iloc[0]['ann_source']
    # extract split_tokens data from .tokens file
    with open(f'data/ebm_nlp_2_00/documents/{id_num}.tokens', 'r') as f:
        tokens = [s.strip() for s in f.readlines()]
    split_tokens = []
    for i in range(len(group.index)):
        row = group.iloc[i]
        rowtext = row['Text'].strip()
        i_tokens = []
        while len(tokens) > 0 and rowtext.find(tokens[0]) == 0:
            i_tokens.append(tokens[0])
            rowtext = rowtext[len(tokens[0]):].strip()
            tokens = tokens[1:]
        split_tokens.append(i_tokens)
    metadata = pd.DataFrame({'split_tokens': split_tokens})
    # extract POS data from .pos file
    with open(f'data/ebm_nlp_2_00/documents/{id_num}.pos', 'r') as f:
        pos = [s.strip() for s in f.readlines()]
    pos_labels = []
    for i in range(len(group.index)):
        sent_tok_len = len(metadata.iloc[i]['split_tokens'])
        pos_labels.append(pos[:sent_tok_len])
        pos = pos[sent_tok_len:]
    metadata['split_tokens_pos'] = pos_labels
    # Extract the direct annotations...
    for annlevel in [('starting_spans', 'ss'), ('hierarchical_labels', 'hl')]:
        for anntype in [('participants', 'p'), ('interventions', 'i'), ('outcomes', 'o')]:
            # read an intersection of (ss/hl)*(p/i/o)
            # (also get annotation clumps while we're doing that)
            try:
                with open(f'data/ebm_nlp_2_00/annotations/aggregated/{annlevel[0]}/{anntype[0]}/{ann_section}/{ann_source}/{id_num}.AGGREGATED.ann', 'r') as f:
                    annotations = [s.strip() for s in f.readlines()]
                ann_col = []
                ann_col_clumps = []
                for i in range(len(group.index)):
                    sent_tok = metadata.iloc[i]['split_tokens']
                    sent_tok_len = len(sent_tok)
                    ann_col.append(annotations[:sent_tok_len])
                    ann_col_clumps.append(extract_labeled_terms(sent_tok, annotations[:sent_tok_len]))
                    annotations = annotations[sent_tok_len:]
                metadata[f'{annlevel[1]}_{anntype[1]}'] = ann_col
                metadata[f'{annlevel[1]}_{anntype[1]}_clumps'] = ann_col_clumps
            except:
                # if the annotation we're looking for doesn't exist, then fill in empty annotations
                metadata[f'{annlevel[1]}_{anntype[1]}'] = [([], [])] * (len(group.index))
                metadata[f'{annlevel[1]}_{anntype[1]}_clumps'] = [([], [])] * (len(group.index))
    for c in join_headers:
        metadata[c] = group[c]
    return metadata

df_metadata = df_sentences.groupby(['ID', 'ann_source'], group_keys=False).apply(
    lambda group: ebmnlp_addtdata(group)
).reset_index(drop=True)

df_metadata.to_pickle(f'temp/ebm-df_metadata.pkl')
df_metadata.iloc[0:100]

In [ ]:
# For now, I'm just going to pretend that EBM-NLP is directly "handling" the issue of splitting
# for us, because we can use the P/I/O annotations as splits!
# ... And if there are no P/I/O annotations, treat it the same as not having a split anchor:
# dump the entire sentence into split_0 and keep all of the anchor-related values to be None/empty

def pio_split(group):
    row = group.iloc[0]
    output = []
    for anntype in ['ss_p']: # ['ss_p', 'ss_i', 'ss_o', 'hl_p', 'hl_i', 'hl_o']:
        clumps, clumps_loc = row[anntype+'_clumps']
        for i in range(len(clumps)):
            # set up split_n
            output_i = [' '.join(row['split_tokens'][:clumps_loc[i]]), 
                        ' '.join(clumps[i]), 
                        ' '.join(row['split_tokens'][clumps_loc[i]+len(clumps[i]):])]
            # split_tokens (copy what we already got as input, haha)
            output_i.append(row['split_tokens'])
            # split_anchor_span
            output_i.append((clumps_loc[i], clumps_loc[i]+len(clumps[i])))
            # split_anchor_indices
            output_i.append((len(output_i[0]), len(output_i[0])+len(output_i[1])))
            # within_anchor_index
            # TODO - perhaps we could assign an actual anchorindex eventually
            # but I'm ignoring that there may be known entity names in PIO annotations for now
            output_i.append(-1)
            # split_anchor_type
            output_i.append(anntype)
            # Add join headers
            output_i += list(row[join_headers])
            output.append(output_i)
    # If there were no annotations, keep the sentence entirely in split_0
    if output == []:
        output = [[' '.join(row['split_tokens']),'','',
                   row['split_tokens'],None,None,None,None]
                  +list(row[join_headers])]
    # Transpose the output format so we can use it in zip for dataframe generation
    output_t = [list(t) for t in list(zip(*output))]
    splits = pd.DataFrame(dict(zip(splitting_headers+join_headers,output_t)))
    return splits

pio_output = df_metadata.groupby(df_metadata.index, group_keys=False).apply(
    lambda group: pio_split(group)
).reset_index(drop=True)

pio_output.to_pickle(f'temp/ebm-pio_output.pkl')
pio_output.to_csv(f'temp/ebm-pio_output.csv')
pio_output.iloc[0:200]

In [ ]:
# Parse the join of df_sentences and df_metadata into something that includes all of the annotations IDed
merged = df_sentences.merge(
    pio_output,
    how='outer',
    left_on=join_headers,
    right_on=join_headers)
merged.iloc[0:200]

In [ ]:
len(df)

In [ ]:
import gensim

# Load Google's pre-trained Word2Vec model.
# model source: https://code.google.com/archive/p/word2vec/
word2vec = gensim.models.KeyedVectors.load_word2vec_format(
    'model/GoogleNews-vectors-negative300.bin', 
    binary=True)

# Get the word2vec embedding of a phrase
def get_phrase_embed_word2vec(word2vec, phrase):
    try:
        phraseS = phrase.split()
    except:
        return pd.DataFrame()
    unknowns = []
    emb = []
    for w in phraseS:
        try:
            emb.append(word2vec[w])
        except:
            unknowns.append(w)
    if len(emb) == 0:
        return pd.DataFrame()
    emb_sum = pd.DataFrame(emb).sum()
    emb_sum['word'] = phrase
    return pd.DataFrame([emb_sum])

v = get_phrase_embed_word2vec(
    word2vec, 
    'test sentence')
sent_v = get_phrase_embed_word2vec(
    word2vec, 
    'This is a test sentence !')
v

In [ ]:
pio_w2v = pio_output.groupby(
    pio_output.index, 
    group_keys=True, 
    as_index=False,
    sort=True
).apply(
    lambda group: get_phrase_embed_word2vec(
        word2vec,
        group.iloc[0]['split_1']
    )
).reset_index(level=1, drop=True)

pio_w2v.iloc[0:200]

In [ ]:
# Visualize the vectors
# loaned from :
# https://github.com/cephcyn/ChatlogGrapher/blob/master/data_processing.ipynb

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import altair as alt
alt.renderers.enable('default')

def visualize_embeds(data, reference, color=None, tooltip=['word']):
    x = data.iloc[:, 0:300]
    x = StandardScaler().fit_transform(x)

    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(
        data=principalComponents,
        columns=['pc1', 'pc2'])

    finalDf = principalDf
    finalDf = finalDf.set_index(data.index)
    finalDf['word'] = data['word']
    finalDf = finalDf.join(
        reference, 
        how='inner',
        lsuffix='_embed', 
        rsuffix='_ref'
    )

    chart = alt.Chart(finalDf).mark_circle(size=60)
    # should figure out a more pythonic way to do this :/
    if color is None:
        chart = chart.encode(
            x='pc1',
            y='pc2',
            tooltip=tooltip
        )
    else:
        chart = chart.encode(
            x='pc1',
            y='pc2',
            color=color,
            tooltip=tooltip
        )
    return chart.interactive()

# visualize_embeds(pd.concat([output_w2v, output_w2v_c]).reset_index())
visualize_embeds(pio_w2v, pio_output)

In [ ]:
# Borrowed from
# https://gmarti.gitlab.io/ml/2017/09/07/how-to-sort-distance-matrix.html

import numpy as np
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
%matplotlib inline

N = len(pio_w2v)
data = pio_w2v.iloc[:, 0:300]
# data = pio_w2v.iloc[:, 0:1024]
# Shuffle data for extra comparison
# data = data.sample(frac=1)
dist_mat = squareform(pdist(data))

# plt.pcolormesh(dist_mat)
# plt.colorbar()
# plt.xlim([0,N])
# plt.ylim([0,N])
# plt.show()

# Borrowed from
# https://gmarti.gitlab.io/ml/2017/09/07/how-to-sort-distance-matrix.html

from fastcluster import linkage

def seriation(Z,N,cur_index):
    '''
        input:
            - Z is a hierarchical tree (dendrogram)
            - N is the number of points given to the clustering process
            - cur_index is the position in the tree for the recursive traversal
        output:
            - order implied by the hierarchical tree Z
            
        seriation computes the order implied by a hierarchical tree (dendrogram)
    '''
    if cur_index < N:
        return [cur_index]
    else:
        left = int(Z[cur_index-N,0])
        right = int(Z[cur_index-N,1])
        return (seriation(Z,N,left) + seriation(Z,N,right))
    
def compute_serial_matrix(dist_mat,method="ward"):
    '''
        input:
            - dist_mat is a distance matrix
            - method = ["ward","single","average","complete"]
        output:
            - seriated_dist is the input dist_mat,
              but with re-ordered rows and columns
              according to the seriation, i.e. the
              order implied by the hierarchical tree
            - res_order is the order implied by
              the hierarchical tree
            - res_linkage is the hierarchical tree (dendrogram)
        
        compute_serial_matrix transforms a distance matrix into 
        a sorted distance matrix according to the order implied 
        by the hierarchical tree (dendrogram)
    '''
    N = len(dist_mat)
    flat_dist_mat = squareform(dist_mat)
    res_linkage = linkage(flat_dist_mat, method=method,preserve_input=True)
    res_order = seriation(res_linkage, N, N + N-2)
    seriated_dist = np.zeros((N,N))
    a,b = np.triu_indices(N,k=1)
    seriated_dist[a,b] = dist_mat[ [res_order[i] for i in a], [res_order[j] for j in b]]
    seriated_dist[b,a] = seriated_dist[a,b]
    
    return seriated_dist, res_order, res_linkage

ordered_dist_mat, res_order, res_linkage = compute_serial_matrix(dist_mat,'ward')

plt.pcolormesh(ordered_dist_mat)
plt.xlim([0,N])
plt.ylim([0,N])
plt.show()

In [ ]:
data = pio_w2v.join(
    pio_output, 
    how='inner'
)
# data = data.loc[pio_output['c_subj_split'] != ''].loc[pio_output['c_verb_split'] != '']
data = data.loc[pio_output['split_1'] != '']

# Shuffle data?
# data = data.sample(frac=1)
dist_mat = squareform(pdist(data.iloc[:, 0:300]))
# dist_mat = squareform(pdist(data.iloc[:, 0:1024]))

ordered_dist_mat, res_order, res_linkage = compute_serial_matrix(
    dist_mat,
    'ward')
reordered_data = data
reordered_data['temp_index'] = data.index
reordered_data = reordered_data.reset_index(drop=True)
reordered_data = reordered_data.iloc[res_order].reset_index(drop=True)
reordered_data['order_cluster'] = reordered_data.index
reordered_data = reordered_data.set_index('temp_index')

reordered_data = pio_output.join(
    reordered_data['order_cluster'], 
    how='outer'
)
reordered_data.to_csv(f'temp/pio-reordered.csv')
reordered_data